<a href="https://colab.research.google.com/github/martda08/Tesis-GPJM/blob/main/4_2_2_Selecci%C3%B3n_del_modelo_(Validaci%C3%B3n_cruzada)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Subsección 4.2.1- Selección del modelo-Modelo Conjunto basado en Campos Gaussianos.

*Fecha de última modificación*: 10-oct-22.

*Tesis*: Modelación de la relación entre el cerebro y el comportamiento mediante campos Gaussianos.

*Autor*: Giwon Bahg

*Modificado por*: Daniela Martínez Aguirre

*Descripción*: Código extraido y modificado del repositorio: https://github.com/MbCN-lab/gpjm y https://github.com/rodrigo-carnier/gpjm.


In [ ]:
#Librerías a utilizar
import numpy as np
#Versión utilizada agosto 2022 2.5.2
!pip install gpflow==2.5.2
import gpflow
import tensorflow as tf
import time
from gpflow.utilities import ops, print_summary  
import pickle



import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Conectar con Google
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Time indices
#Importa los y los tiempos
time_block = np.load("/content/drive/MyDrive/Tesis/Cap4/fMRI_DATOS/time_block.npy")# Duración de cada bloque(son 4 bloques y cada bloque tiene 20 ensayos)
ts_dense = np.load("/content/drive/MyDrive/Tesis/Cap4/fMRI_DATOS/ts_dense.npy") # Time indices for behavioral data-cada medio segundo hasta 784
ts_sparse = np.load("/content/drive/MyDrive/Tesis/Cap4/fMRI_DATOS/ts_sparse.npy") # Time indices for neural data -cada segundo desde 0 hasta 783

In [ ]:
#Time indices for the fixation onset (i.e., beginning of new trials)
#Cada cuando empieza cada uno de los 20 ensayos
#Va de 39 en 39 hasta 744
fix_onset = np.array([  0.,  39.,  78., 117., 156., 195., 235., 274., 313., 353., 393., 431., 469., 508., 547., 587., 627., 665., 705., 744.], dtype=np.int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  after removing the cwd from sys.path.


In [ ]:
#Load the coherence information
#Sólamente se importa un bloque
#Only the variable 'coh1', which corresponds to the first run of the experiment, is used.
coh1 = np.load("/content/drive/MyDrive/Tesis/Cap4/fMRI_DATOS/coherence_scaled_013_run1.npy")

#Upscale the coherence
coh1_ext = np.column_stack([coh1, coh1]).ravel()
coh1_ext = coh1_ext[0:(len(coh1_ext)-1)]

#La coherencia es reescalada de -1 a 1 en lugar de -.35 a .35
coh1 = coh1_ext.reshape(-1,1)

In [ ]:
#Importar datos de comportamiento
# Load the mouse trajectory
# Only the variable 'Y_B1' is used.
Y_B1_ori = np.load("/content/drive/MyDrive/Tesis/Cap4/fMRI_DATOS/mouse_trajectory_centered_scaled_013_run1.npy")

#Logit-transform the behavioral data
temp = Y_B1_ori/2 + 0.5
temp[np.where(temp == 0)[0]] = 1e-5
Y_B1 = np.log(temp/(1-temp))

#Logit-transform the coherence
temp = coh1/2 + 0.5
temp[np.where(temp == 0)[0]] = 1e-5
coh1_transformada= np.log(temp/(1-temp))

Y_B1_ori = np.load("/content/drive/MyDrive/Tesis/Cap4/fMRI_DATOS/mouse_trajectory_centered_scaled_013_run1.npy")


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in true_divide
  


In [ ]:
# Load the neural data-sólo el primer bloque
Y_N1 = np.load("/content/drive/MyDrive/Tesis/Cap4/fMRI_DATOS/Timing013_meanTS_block1.npy")
# Y_N2 = np.load("Timing013_meanTS_block2.npy") # Not used
# Y_N3 = np.load("Timing013_meanTS_block3.npy")
# Y_N4 = np.load("Timing013_meanTS_block4.npy")

In [ ]:
# ROI information: Not used for fitting the model.
## Coherence-related regions
ss_RC = np.array([[-54, 23, 0], [32, -23, 64], [29, -69, -41], [-48, 48, -14], [-9, -88, 26], [49, -40, 9], [7, -72, -9]], dtype = np.float64)
## Behavioral-response-related regions
ss_resp = np.array([[66, -3, 22], [54, -14, 6], [-44, -64, 11], [10, -6, 72], [-44, -3, 61], [-52, 0, 32], [60, -25, 13], [-53, -21, 10], [-10, -56, 68]], dtype = np.float64)

ss = np.vstack([ss_RC, ss_resp])

# ROI labels
label_ROI = np.array(['RC1_IFG', 'RC2_PCG', 'RC3_CRUS2', 'RC4_FP', 'RC5_OP', 'RC6_PSG', 'RC7_LG',
            'resp1_SSC1', 'resp2_AC1', 'resp3_V5', 'resp4_PMC(SFG)', 'resp5_PMC(MFG)','resp6_MC1', 'resp7_IPL', 'resp8_AC1', 'resp9_SPL'])

#Definir el modelo

In [ ]:
#Creación de modelo: 
#Establece las funciones kernel a utilizar
#Inicializa la variable latente
#Construye la función objetivo
class GPJMv3(gpflow.models.GPR):
    def __init__(self, Y_N, Y_B, ts_N, ts_B, n_latent, ss,
                 kern_tX = None, mean_tX = None, kern_XN = None, mean_XN = None, kern_XB = None, mean_XB = None, name=None):
        if kern_tX is None:
            kern_tX = gpflow.kernels.Matern12() 
        if mean_tX is None:
            mean_tX = gpflow.mean_functions.Zero(output_dim =n_latent)
        if kern_XN is None:
            kern_XN =gpflow.kernels.SquaredExponential() 
        if mean_XN is None:
            mean_XN = gpflow.mean_functions.Zero(output_dim = Y_N.shape[1]) 
        if kern_XB is None:
            kern_XB = gpflow.kernels.Matern12()
        if mean_XB is None:
            mean_XB = gpflow.mean_functions.Zero(output_dim = Y_B.shape[1])
       # super().__init__(name=name)
        
        def cubic_interpolation(ts_N, Y_N, ts_B, ss):
            from scipy import interpolate
            yn_new = np.zeros((ts_B.shape[0], ss.shape[0]))
            yn_array = Y_N.reshape(ss.shape[0], ts_N.shape[0]).T
            for i in range(ss.shape[0]):
                temp = interpolate.interp1d(np.squeeze(ts_N), yn_array[:,i], kind='cubic',  fill_value="extrapolate")
                yn_new[:,i] = temp(np.squeeze(ts_B))
            return yn_new
        
        def downsizing_scheme_nearest(ts_N, ts_B):
            M = np.zeros((ts_B.shape[0], ts_N.shape[0]))
            ts = np.squeeze(ts_B)
            for i in range(ts_N.shape[0]):
                argmin_idx = np.argmin(np.abs(ts - ts_N[i,0]))
                M[argmin_idx,i] = 1
            return M
        
        def HRF_filter(ts_B):
            ts = np.squeeze(ts_B)
            unit_ts = ts[ts <= 30]
            def HRFunit(t):
                from scipy.special import gamma
                a1 = 6 # b1=1
                a2 = 16 # b2=1
                c = 1./6
                part1 = t**(a1-1) * np.exp(-t) / gamma(a1)
                part2 = t**(a2-1) * np.exp(-t) / gamma(a2)
                return part1 - c * part2
            hrf = HRFunit(unit_ts)
            return(hrf)
        
        if len(ts_N) > len(ts_B):
            print("Neural: Dense / Behavioral: Sparse")
            self.ts = tf.constant(ts_N.copy())
        elif len(ts_N) < len(ts_B):
            print("Neural: Sparse / Behavioral: Dense")
            self.ts = tf.constant(ts_B.copy())
            self.ts_B = ts_B.copy()
            self.Y_N_interp = cubic_interpolation(ts_N, Y_N, ts_B, ss)
            self.M = downsizing_scheme_nearest(ts_N, ts_B)
        
        # Data
        self.Y_N = tf.constant(Y_N.copy())
        self.Y_B = tf.constant(Y_B.copy())
        self.ts_N = tf.constant(ts_N.copy())
        self.ts_B = tf.constant(ts_B.copy())
        self.n_Nsample = Y_N.shape[0]
        self.n_Nfeature = Y_N.shape[1]
        self.n_Bsample = Y_B.shape[0]
        self.n_Bfeature = Y_B.shape[1]
        
        # latent dynamics kernel + downsizing scheme
        self.kern_tX = kern_tX
        self.mean_tX = mean_tX
        self.n_latent = n_latent
        self.N_pca = ops.pca_reduce(self.Y_N_interp, n_latent)
        self.X = gpflow.Parameter(ops.pca_reduce(self.Y_N_interp, n_latent))
        self.X_sparse = ops.pca_reduce(Y_N, n_latent)

        # Neural data kernel
        self.kern_XN = kern_XN
        self.mean_XN = mean_XN
        self.hrf = tf.constant(HRF_filter(self.ts_B))
        
        # Behavioral data kernel
        self.kern_XB = kern_XB
        self.mean_XB = mean_XB

        # Likelihood
        self.likelihood_tX = gpflow.likelihoods.Gaussian()
        self.likelihood_XN = gpflow.likelihoods.Gaussian()
        self.likelihood_XB = gpflow.likelihoods.Gaussian() # Can differ according to the model you rely on.
    
    #@gpflow.params_as_tensors
    def _build_likelihood_tX(self): # Zero-noise model is not supported by GPflow ==> Need to add an infinitesimal noise when initializing the model.
        Ktx = self.kern_tX.K(self.ts, self.ts) + tf.eye(tf.shape(self.ts)[0], dtype =tf.dtypes.float64) * self.likelihood_tX.variance
        Ltx = tf.linalg.cholesky(Ktx)
        mtx = self.mean_tX(self.ts)
        logpdf_tx = gpflow.logdensities.multivariate_normal(self.X, mtx, Ltx)
        return tf.reduce_sum(logpdf_tx)
    
    #@gpflow.params_as_tensors
    def _build_likelihood_XN(self):
        kernel_completo=self.kern_XN.K(self.X, self.X) 
        kernel_incompleto=tf.matmul(tf.transpose(self.M), tf.matmul(kernel_completo, self.M))
        Kxn = kernel_incompleto + tf.eye(self.n_Nsample, dtype =tf.dtypes.float64) * self.likelihood_XN.variance
        Lxn = tf.linalg.cholesky(Kxn)  
        mxn = self.mean_XN(self.X_sparse)
        logpdf_xn = gpflow.logdensities.multivariate_normal(self.Y_N, mxn, Lxn)
        return tf.reduce_sum(logpdf_xn)

    #@gpflow.params_as_tensors
    def _build_likelihood_XB(self):
        Kxb = self.kern_XB.K(self.X, self.X) + tf.eye(tf.shape(self.X)[0], dtype = tf.dtypes.float64) * self.likelihood_XB.variance
        Lxb = tf.linalg.cholesky(Kxb)
        mxb = self.mean_XB(self.X)
        logpdf_xb = gpflow.logdensities.multivariate_normal(self.Y_B, mxb, Lxb)
        return tf.reduce_sum(logpdf_xb)


    def maximum_log_likelihood_objective(self):     # 2022-02 RMC upd03: Abstract function for calculating log_likelihood now is named like this. (Was this the purpose of this function "_build_likelihood"?)
        with tf.name_scope('likelihood') as scope:  # 2022-02 RMC upd02: This is how name_scopes are defined nowaways.
            logpdf_tx = self._build_likelihood_tX()
            logpdf_xn = self._build_likelihood_XN()
            logpdf_xb = self._build_likelihood_XB()
            return tf.reduce_sum(logpdf_tx + logpdf_xn + logpdf_xb)
   

In [ ]:
#Funciones para hacer predicciones de datos nuevos

def recover_Kxn(m, ts_new):
    X_new = recover_latent(m, ts_new)
    X = m.X.numpy()
    ss =m.ss.numpy()
    kern_temporal = m.kern_XN.kernel_t(X, X_new).numpy()
    kern_spatial = m.kern_XN.kernel_s(ss, ss).numpy()
    ts_N =m.ts_N.numpy()
        
    i, k, s = ss.shape[0], ts_N.shape[0], ts_N.shape[0]
    o = s * (i - 1) + k
  
    Kss = tf.reshape(kern_spatial, [1, i, i, 1])
    Ktt = tf.reshape(kern_temporal, [k, k, 1, 1])
    Kst = tf.squeeze(tf.nn.conv2d_transpose(Kss, Ktt, (1, o, o, 1), [1, s, s, 1], "VALID"))
    return Kst

#Recuperar la variable latente
def recover_latent(m, ts_new):
    import tensorflow as tf
    from numpy.linalg import inv, cholesky
    ts = m.ts.numpy()
    Kstar = m.kern_tX(ts, ts_new).numpy()
    KttI = (m.kern_tX(ts, ts) + np.eye(ts.shape[0], dtype=np.float64) * m.likelihood_tX.variance).numpy()
    X = m.X.numpy()
    L = cholesky(KttI)
    return Kstar.T.dot(inv(L.T).dot((inv(L)).dot(X)))

#Recuperar los datos neuronales
def recover_neural(m, ts_new):
    import tensorflow as tf
    from numpy.linalg import inv, cholesky
    m=test
    ts_N =m.ts_N.numpy()
    Y_N = m.Y_N.numpy()
    X_new = recover_latent(m, ts_new)
    X = m.X.numpy()[train_index % 2 == 0]
    Kstar=m.kern_XN(X, X_new).numpy()
    KttI = (m.kern_XN(X, X) + np.eye(len(tss_train), dtype = np.float64) * m.likelihood_XN.variance).numpy()
    L = cholesky(KttI)
    fmean = Kstar.T.dot(inv(L.T).dot((inv(L)).dot(Y_N)))
    v = inv(L).dot(Kstar)
    Vstar = m.kern_XN(X_new, X_new).numpy() - v.T.dot(v)+np.eye(X_new.shape[0], dtype = np.float64) * m.likelihood_XN.variance
    sd = np.sqrt(np.diag(Vstar))
    return fmean, Vstar, sd
    
#Recuperar los datos de comportamiento
def recover_behavioral(m, ts_new):
    import tensorflow as tf
    from numpy.linalg import inv, cholesky
    X_new = recover_latent(m, ts_new)
    X = m.X.numpy()
    Kstar = m.kern_XB(X, X_new).numpy()
    KttI = (m.kern_XB(X, X) + np.eye(len(tsd_train), dtype = np.float64) * m.likelihood_XB.variance).numpy()
    Y_B = m.Y_B.numpy()
    L = cholesky(KttI)
    fmean = Kstar.T.dot(inv(L.T).dot((inv(L)).dot(Y_B)))
    v = inv(L).dot(Kstar)
    Vstar = m.kern_XB(X_new, X_new).numpy() - v.T.dot(v)+np.eye(X_new.shape[0], dtype = np.float64) * m.likelihood_XB.variance
    sd= np.sqrt(np.diag(Vstar))
    ci95 = np.column_stack([fmean - 1.96 * np.sqrt(np.diag(Vstar)).reshape(-1,1), fmean + 1.96 * np.sqrt(np.diag(Vstar)).reshape(-1,1)])
    return fmean, Vstar, ci95


#Ajuste del modelo

In [ ]:
#Centrar y escalar los datos

#Hacemos una copia de los datos
Y_N1_es=Y_N1.copy()
Y_B1_es=Y_B1.copy()

#Estandarizamos los datos neuronales
Y_N1_es-= np.mean(Y_N1_es, axis=0)
Y_N1_es /= np.std(Y_N1_es, axis=0)

#Estandarizamos los datos de comportamiento
Y_B1_es-= np.mean(Y_B1_es, axis=0)
Y_B1_es /= np.std(Y_B1_es, axis=0)

#with open('ECM', 'rb') as ECM:
#    dataset = pickle.load(ECM)

#with open('log', 'rb') as log:
 #   dataset = pickle.load(log)

In [ ]:
#Optimización del modelo usando descenso por gradiente
import numpy as np
import numpy as np
from sklearn.model_selection import KFold
import tensorflow as tf
from numpy.linalg import inv, det
import math as math

#número de iteraciones
max_iter = 700
record_step = 20

#Validación cruzada n_splits-fold
n_splits=10
latentes=7
kf = KFold(n_splits=n_splits,shuffle=True, random_state=1)

kf.get_n_splits(np.arange(1567))
j=0
ECM=np.zeros((n_splits,latentes-1))
#ECM2=np.zeros((n_splits,latentes-1))
log=np.zeros((n_splits,latentes-1))
#log2=np.zeros((n_splits,latentes-1))

#Ciclo para ajustar el modelo para cada muestra de la validación cruzada y para distintas dimensiones de la variable latente
for train_index, test_index in kf.split(np.arange(1567)):
  print("TRAIN:", train_index, "TEST:", test_index)
  tsd_train, tsd_test = ts_dense[train_index,:], ts_dense[test_index,:]
  tss_train, tss_test = ts_dense[train_index[train_index % 2 == 0],:], ts_dense[test_index[test_index % 2 == 0] ,:]
  for n_latent in range(1,latentes):
    print(n_latent)#Agregamos datos al modelo
    r=n_latent-1
    Y_N1_train=Y_N1_es[tss_train.astype(int)[:,0]]
    Y_B1_train=Y_B1_es[train_index, :]
    Y_N1_test=Y_N1_es[tss_test.astype(int)[:,0]]
    Y_B1_test=Y_B1_es[test_index, :]
    test= GPJMv3(Y_N1_train ,Y_B1_train, tss_train, tsd_train, n_latent, ss)
    #test.likelihood_tX.variance = 1e-6
    #gpflow.set_trainable(test.likelihood_tX, False)
    
    @tf.function
    def optimisation_step():
      adam_opt.minimize(test.training_loss, test.trainable_variables)


    X_storage = np.zeros((test.X.shape[0], test.X.shape[1], (max_iter+1)))
    llk_storage = np.zeros(max_iter+1)
    learning_rate_vec_temp = 0.25/np.sqrt(np.arange(max_iter)+1)
    learning_rate_vec = np.zeros_like(learning_rate_vec_temp)
    for i in range(len(learning_rate_vec_temp)):
      learning_rate_vec[i] = learning_rate_vec_temp[int(np.floor(i/record_step))] 

    for iter in range(max_iter):
      l = learning_rate_vec[iter]
      adam_opt = tf.optimizers.Adam(l)
      optimisation_step()

      if max_iter % record_step== 0 and iter > 0:
        print(f"Epoch {iter} -kfold {j}-n_latent {n_latent} Loss: {test.training_loss().numpy() : .4f}")


    #Calculo de la log-probabilidad

    yhat_arr, yhat_v, yhat_sd = recover_neural(test, tss_test)
    bmean, bV, bci = recover_behavioral(test, tsd_test)
    
    #calculo del error cuadrático médio
    k1=len(tss_test)
    k2=len(tsd_test)
    k3=16
    ECM[j,r]=(np.sum((yhat_arr-Y_N1_test)**2)+np.sum((bmean-Y_B1_test)**2))/(k1*k3+k2)
   
    #GUardamos los errores en una tabla
    with open('ECM', 'wb') as output:
      pickle.dump(ECM, output)
    lxn=tf.linalg.cholesky(bV)
    log2=gpflow.logdensities.multivariate_normal(Y_B1_test, bmean, lxn).numpy()
    lxn=tf.linalg.cholesky(yhat_v)
    log3=np.sum(gpflow.logdensities.multivariate_normal(yhat_arr,Y_N1_test, lxn).numpy())
    log[j,r]=log2+log3
    with open('LOG', 'wb') as output:
      pickle.dump(log, output)


  j=j+1
  

Streaming output truncated to the last 5000 lines.
Epoch 664 -kfold 0-n_latent 3 Loss:  6761.9543
Epoch 665 -kfold 0-n_latent 3 Loss:  6740.4040
Epoch 666 -kfold 0-n_latent 3 Loss:  6749.2907
Epoch 667 -kfold 0-n_latent 3 Loss:  6802.2947
Epoch 668 -kfold 0-n_latent 3 Loss:  6840.1441
Epoch 669 -kfold 0-n_latent 3 Loss:  6796.6891
Epoch 670 -kfold 0-n_latent 3 Loss:  6766.4414
Epoch 671 -kfold 0-n_latent 3 Loss:  6684.0386
Epoch 672 -kfold 0-n_latent 3 Loss:  6629.2447
Epoch 673 -kfold 0-n_latent 3 Loss:  6597.4525
Epoch 674 -kfold 0-n_latent 3 Loss:  6489.8220
Epoch 675 -kfold 0-n_latent 3 Loss:  6496.9930
Epoch 676 -kfold 0-n_latent 3 Loss:  6483.0923
Epoch 677 -kfold 0-n_latent 3 Loss:  6533.1598
Epoch 678 -kfold 0-n_latent 3 Loss:  6594.6803
Epoch 679 -kfold 0-n_latent 3 Loss:  6550.4849
Epoch 680 -kfold 0-n_latent 3 Loss:  6553.5324
Epoch 681 -kfold 0-n_latent 3 Loss:  6552.5662
Epoch 682 -kfold 0-n_latent 3 Loss:  6434.2920
Epoch 683 -kfold 0-n_latent 3 Loss:  6381.4567
Epoch 684